In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csc_matrix as csc
import pandas as pd

In [2]:
LOG_DIR = r'C:\TensorLogs'
NAME = r'BasicMatrixFactorization' 
DATA_DIR = r'./data/'
Y_HAT_PATH = DATA_DIR+r'/'+NAME+r'-y_hat.npz'
TEST_DATA_PATH = DATA_DIR+r'uxm_test.npz'
TEST_RESULTS_PATH = LOG_DIR+'\\'+NAME+'\\test-results.csv'
DATA_OCT = DATA_DIR+r'2019-Oct.csv'
DATA_NOV = DATA_DIR+r'2019-Nov.csv'
USECOLS = ["event_type","product_id","user_id"]

In [3]:
ux_constants = pd.Series(pd.read_csv(DATA_DIR+r'ux_constants.csv', index_col=0, squeeze=True, header=None), dtype='float32')
VIEW     = ux_constants['view_to_purchase']
CART     = ux_constants['cart_to_purchase']
REMOVE   = ux_constants['remove_to_purchase']
PURCHASE = ux_constants['purchase_to_purchase']

ParserError: Error tokenizing data. C error: Expected 1 fields in line 2, saw 2


In [ ]:
log = pd.Series(dtype='float64')
y_hat = sp.load_npz(Y_HAT_PATH)
y = sp.load_npz(TEST_DATA_PATH)
assert y_hat.shape == y.shape, 'The shape of Y and Y_hat must match, otherwise they are not comparable.'
print(f"Shape of the matrices: {y.shape}")
print("Number of non-zero values:")
print(f"Y: {y.nnz:8,}")
print(f"Ŷ: {y_hat.nnz:8,}")

In [ ]:
# Usually, the CSC is used when there are more rows than columns. (If there are more columns, use CSR instead.)
y_hat = y_hat.tocsc()
y = y.tocsc()

In [ ]:
MSE = csc.sum(csc.power(y_hat-y,2))/y.nnz
print(f"Mean Square Error: {MSE}")
log["MSE"]=MSE

In [ ]:
RMSE = np.sqrt(MSE)
print(f"Root Mean Square Error: {RMSE}")
log["RMSE"]=RMSE

In [ ]:
MAE = csc.sum(abs(y_hat-y))/y.nnz
print(f"Mean Absolute Error: {MAE}")
log["MAE"]=MAE

In [ ]:
log

In [ ]:
log.to_csv(TEST_RESULTS_PATH, index = True, header=False)